# Aaronson-Rall

- Be able to see internal parameters throughout the algorithm 
- Ratio of t_min, t_max 


In [12]:
import matplotlib.pyplot as plt
import numpy as np

from random import sample, seed

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile, assemble
from qiskit.quantum_info import Statevector
from qiskit.algorithms import amplitude_estimators, EstimationProblem
from qiskit.extensions import UnitaryGate

from qiskit.visualization import plot_histogram

from algorithms.amplitude_estimators.aes import AmplitudeEstimationSimplified as AES
from operators import *


In [2]:

sim = Aer.get_backend('aer_simulator')  # Tell Qiskit how to simulate our circuit
qc = QuantumCircuit(1, 1)
qc.ry(2 * 1.56979632663, 0)

qc.save_statevector()
qobj = assemble(qc)
state = sim.run(qobj).result().get_statevector()

print(state) # Display the output state vector


[0.001    +0.j 0.9999995+0.j]


/usr/local/lib/python3.9/site-packages/qiskit/utils/deprecation.py:62: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  return func(*args, **kwargs)


In [3]:
n = 4
N = 2**n
k = N//2
marked = sample(range(N), k)

## Define the estimation problem 
Qiskit's amplitude estimators have the same interface, so the same definition of the estimation problem can be reused.

https://qiskit.org/documentation/stubs/qiskit.algorithms.EstimationProblem.html#qiskit.algorithms.EstimationProblem

In [4]:
# Define the estimation problem
# https://qiskit.org/documentation/stubs/qiskit.algorithms.EstimationProblem.html#qiskit.algorithms.EstimationProblem
def good_state(state):
    bin_marked = [(n-len(bin(s))+2)*'0'+bin(s)[2:] for s in marked]
    return (state in bin_marked)

problem = EstimationProblem(
    state_preparation=A(n),  # A operator
    grover_operator=Q(n, marked),  # Q operator
    objective_qubits=range(n),
    is_good_state=good_state  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

In [5]:
num_qubits = max(
            problem.state_preparation.num_qubits,
            problem.grover_operator.num_qubits,
        )
circuit = QuantumCircuit(num_qubits+2, name="circuit")

# Convert marked states to bits      7 -> 0111 
# For each marked state, create mcx gate controlled by corresponding bits 
A_ = problem.state_preparation

# AR
AR_qc = QuantumCircuit(num_qubits+2)
AR_qc.append(A_, [0,1,2,3])
marked_s = sorted(marked, reverse=True)
for target in range(16):
    if marked_s and marked_s[-1] == target:
        marked_s.pop()
    else:
        if not target & 0b1000:
            AR_qc.x(3)
        if not target & 0b0100:
            AR_qc.x(2)
        if not target & 0b0010:
            AR_qc.x(1)
        if not target & 0b0001:
            AR_qc.x(0)
        AR_qc.mcx([0,1,2,3], 4)
        if not target & 0b0001:
            AR_qc.x(0)
        if not target & 0b0010:
            AR_qc.x(1)
        if not target & 0b0100:
            AR_qc.x(2)
        if not target & 0b1000:
            AR_qc.x(3)
AR_qc.u(3.13959465126, 0, 0, 5)  # R 
AR_gate = AR_qc.to_gate(label="A x R")
# END AR 

AR_gate_inv = AR_gate.inverse()

# ZERO GATE
zero_vec = np.zeros(2**(n+2))
zero_vec[0] = 1
zero_gate = UnitaryGate(np.eye(2**(n+2)) - 2 * np.tensordot(zero_vec, zero_vec, axes=0))
# END ZERO GATE

# LAST GATE 
measure_vec = np.zeros(2**2)
measure_vec[0] = 1
measure_gate = UnitaryGate(-1 * (np.eye(2**(n+2)) - 2 * (np.tensordot(np.eye(2**n), np.tensordot(measure_vec, measure_vec, axes=0), axes=0)).swapaxes(1, 2).reshape(2**(n+2), 2**(n+2))))
# END LAST GATE

# G OPERATOR
G_qc = QuantumCircuit(num_qubits+2)
G_qc.append(measure_gate, range(6))
G_qc.append(AR_gate_inv, range(6))
G_qc.append(zero_gate, range(6))
G_qc.append(AR_gate, range(6))
G_gate = G_qc.to_gate(label="G")
# END G OPERATOR



circuit.append(AR_gate, [0,1,2,3,4,5])
# circuit.append(A_.inverse(), [0,1,2,3])
sv = Statevector(circuit)
D = sv.to_dict()
print(sorted(marked))
# for k in D:
#     if k[0:2] == '01':
#         print(k, D[k])
D

[2, 3, 7, 8, 9, 10, 13, 15]


{'000010': (0.24999999999999992+0j),
 '000011': (0.24999999999999992+0j),
 '000111': (0.24999999999999992+0j),
 '001000': (0.24999999999999992+0j),
 '001001': (0.24999999999999992+0j),
 '001010': (0.24999999999999992+0j),
 '001101': (0.24999999999999992+0j),
 '001111': (0.24999999999999992+0j),
 '010000': (0.24999999999999992+0j),
 '010001': (0.24999999999999992+0j),
 '010100': (0.24999999999999992+0j),
 '010101': (0.24999999999999992+0j),
 '010110': (0.24999999999999992+0j),
 '011011': (0.24999999999999992+0j),
 '011100': (0.24999999999999992+0j),
 '011110': (0.24999999999999992+0j)}

In [6]:
# use local simulator
aer_sim = Aer.get_backend('aer_simulator')
aer_sim._configuration.max_shots = 300


In [8]:
aes = AES(0.1, 0.1, marked=marked, quantum_instance=aer_sim)

In [9]:
aes.construct_circuit(problem, 3)

In [13]:
aes.estimate(problem)

# n_trials = 100

# max_depth_log2 = 3
# queries = np.zeros((max_depth_log2-1, n_trials))
# errs = np.zeros((max_depth_log2-1, n_trials))

# for i in range(2,max_depth_log2+1):
#     for j in range(n_trials):
#         mlae = MLAE(range(i), quantum_instance=aer_sim)
#         result = mlae.estimate(problem)
#         queries[i-2,j] = result.num_oracle_queries
#         errs[i-2,j] = abs(k/N - result.estimation)

AttributeError: 'AmplitudeEstimationSimplified' object has no attribute '_evaluation_schedule'

In [ ]:
plt.scatter(queries.mean(axis=1), errs.mean(axis=1))
plt.plot(queries.mean(axis=1), errs.mean(axis=1))
plt.yscale('log')
plt.xscale('log')
plt.xlim(1, 10**10)
plt.show()